In [1]:
# import qiskit 
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import GroverOperator
from qiskit.circuit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2  as Sampler
# Initialize the Qiskit Runtime Service
from test_utils import read_account_file

In [ ]:
def grover_search(n, oracle):
    """
    Perform Grover's search algorithm.

    Parameters:
    n (int): Number of qubits.
    oracle (QuantumCircuit): Oracle circuit that marks the solution.

    Returns:
    QuantumCircuit: The Grover search circuit.
    """
    # Create quantum and classical registers
    qr = QuantumRegister(n, 'q')
    cr = ClassicalRegister(n, 'c')
    
    # Create the Grover circuit
    grover_circuit = QuantumCircuit(qr, cr)
    
    # Initialize qubits to |+>
    for i in range(n):
        grover_circuit.h(qr[i])
    
    # Apply the oracle
    grover_circuit.compose(oracle,inplace=True)
    
    # Apply the Grover diffusion operator
    grover_circuit.append(GroverOperator(n).to_instruction(), qr)
    
    # Measure the qubits
    grover_circuit.measure(qr, cr)
    
    return grover_circuit

In [3]:
def run_grover_search(backend,n, oracle):
    """
    Run Grover's search algorithm and return the results.

    Parameters:
    n (int): Number of qubits.
    oracle (QuantumCircuit): Oracle circuit that marks the solution.

    Returns:
    dict: Measurement results.
    """
    # Create the Grover search circuit
    grover_circuit = grover_search(n, oracle)
    
    pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
    # Transpile the circuit
    transpiled_circuit = pm.run(grover_circuit)
    transpiled_circuit.draw("mpl")  # Draw the transpiled circuit
    # Create a sampler instance
    sampler = Sampler(mode=backend)
    # Sample the transpiled circuit
    job = sampler.run([transpiled_circuit], shots=1024)
    
    return job

In [9]:
# Example usage
if __name__ == "__main__":
    # Define the oracle for a simple case (e.g., marking |11>)
    n = 2  # Number of qubits
    oracle = QuantumCircuit(n)
    oracle.x(0)  # Marking |11> state
    oracle.x(1)
    oracle.h(0)
    oracle.h(1)
    oracle.cz(0, 1)  # Apply controlled-Z gate
    oracle.h(0)
    oracle.h(1)
    oracle.x(0)
    oracle.x(1)

    account = read_account_file('account_info.conf')
    service = QiskitRuntimeService(channel="ibm_cloud", instance=account['instance'], token=account['token'])
    backend = service.least_busy()
    print(f"Using backend: {backend.name}")

    # Run Grover's search
    
    job = run_grover_search(backend,n, oracle)
    
    print(f"Job ID: {job.job_id()}")    

Using backend: ibm_torino


AttributeError: 'int' object has no attribute 'num_qubits'

In [ ]:
result = job.result()

# Get the counts (measurement results)
counts = results[0].data.c.get_counts()

# Print the results
print("Measurement results:", results) 
# Optionally, visualize the results
plot_histogram(results).show()